In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Add, Dense, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

class StopAtThresholdCallback(Callback):
    def __init__(self, threshold):
        super(StopAtThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_loss = logs.get('val_loss')
        if val_loss is not None and val_loss < self.threshold:
            print(f"\nStopping training as validation loss {val_loss} is below the threshold {self.threshold}")
            self.model.stop_training = True

callback = StopAtThresholdCallback(threshold=1e-07)

class H1Layer(Layer):
    def __init__(self, **kwargs):
        super(H1Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.b = self.add_weight(shape=(input_shape[-1],),
                                 initializer='random_normal',
                                 trainable=True)
        super(H1Layer, self).build(input_shape)

    def call(self, x):
        return self.b * (2 * x)
        #return (2 * x) 


class H2Layer(Layer):
    def __init__(self, h1, **kwargs):
        super(H2Layer, self).__init__(**kwargs)
        self.h1 = h1

    def call(self, x):
        return (2*x*(self.h1(x)))-2
    
class H3Layer(Layer):
    def __init__(self, h1, h2, **kwargs):
        super(H3Layer, self).__init__(**kwargs)
        self.h1 = h1
        self.h2 = h2
        
    def call(self, x):
        return (2*x*(self.h2(x)))-(4*self.h1(x))

class H4Layer(Layer):
    def __init__(self, h2, h3, **kwargs):
        super(H4Layer, self).__init__(**kwargs)
        self.h2 = h2
        self.h3 = h3

    def call(self, x):
        return (2*x*(self.h3(x)))-(6*self.h2(x))   
    
class TensorDecompositionLayer(Layer):
    def __init__(self, rank, **kwargs):
        self.rank = rank
        super(TensorDecompositionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.factors_a = self.add_weight(shape=(input_shape[-1], self.rank),
                                         initializer='random_normal',
                                         trainable=True)
        self.factors_b = self.add_weight(shape=(self.rank, input_shape[-1]),
                                         initializer='random_normal',
                                         trainable=True)
        super(TensorDecompositionLayer, self).build(input_shape)

    def call(self, x):
        return tf.matmul(tf.matmul(x, self.factors_a), self.factors_b)

# Define the function
def f(x1, x2):
    return 3 * np.cos(2 * np.pi * (x1**2 - x2**2))

# Set the parameters
lower_bound = -3
upper_bound = 3
n_samples = 1000

# Generate the data
x1_values = np.linspace(lower_bound, upper_bound, n_samples).reshape(n_samples, 1)
x2_values = np.linspace(lower_bound, upper_bound, n_samples).reshape(n_samples, 1)

# Get a meshgrid for x1 and x2 values
X1, X2 = np.meshgrid(x1_values, x2_values)

# Calculate y values using the function
y_values = f(X1, X2)

# Reshape the data for training
X = np.column_stack((X1.ravel(), X2.ravel()))
y = y_values.ravel()


from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# X[:-5], y[:-5]  # Displaying the first 5 samples


2023-10-17 12:31:22.937547: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-17 12:31:23.400647: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

def build_model(input_shape, filters):
    input_layer = Input(shape=input_shape)
    x = input_layer

    x = Dense(filters, activation="relu")(x)
    x = Dense(filters, activation="relu")(x)
    x = Dense(filters, activation="relu")(x)
    x = Dense(filters, activation="relu")(x)
    
    output_layer = Dense(1)(x)
    model = Model(inputs=input_layer, outputs=output_layer)

    return model 


input_shape = (2,)
filters = 64
modelN4 = build_model(input_shape, filters)
optimizer = Adam(learning_rate=0.001) # Reduce learning rate
modelN4.compile(optimizer=optimizer, loss='mse')

batch_size = 64
epochs = 150

history = modelN4.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_val, y_val),
                    callbacks=[callback])

val_loss = modelN4.evaluate(X_val, y_val, verbose=0)
print(f"Validation loss: {val_loss}")

import matplotlib.pyplot as plt

num_test_samples = 100
x1_test_values = np.linspace(lower_bound, upper_bound, num=num_test_samples).reshape(-1, 1)
x2_test_values = np.linspace(lower_bound, upper_bound, num=num_test_samples).reshape(-1, 1)

# Get a meshgrid for x1 and x2 test values
X1_test, X2_test = np.meshgrid(x1_test_values, x2_test_values)

# Calculate y true values using the function
y_true_3d = f(X1_test, X2_test)

X_test_3d = np.column_stack((X1_test.ravel(), X2_test.ravel()))
y_true_3d = y_true_3d.ravel()


# 3D visualization of the data
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot
ax.scatter(X_test_3d[:, 0], X_test_3d[:, 1], y_true_3d, c=y_true_3d, cmap='viridis', marker='o')

ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('y_true_3d')
ax.set_title('3D Mapping of Function')
ax.grid(True)

plt.show()


Epoch 1/150


2023-10-17 12:31:24.389736: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-17 12:31:24.406516: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-17 12:31:24.406718: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

12500/12500 [==============================] - 26s 2ms/step - loss: 3.3049 - val_loss: 2.8005
Epoch 2/150
12500/12500 [==============================] - 24s 2ms/step - loss: 2.7070 - val_loss: 2.3996
Epoch 3/150
12500/12500 [==============================] - 24s 2ms/step - loss: 2.3158 - val_loss: 2.1907
Epoch 4/150
12500/12500 [==============================] - 24s 2ms/step - loss: 2.1159 - val_loss: 2.0475
Epoch 5/150
12500/12500 [==============================] - 24s 2ms/step - loss: 2.0021 - val_loss: 1.9762
Epoch 6/150
12500/12500 [==============================] - 24s 2ms/step - loss: 1.8233 - val_loss: 1.6804
Epoch 7/150
12500/12500 [==============================] - 24s 2ms/step - loss: 1.6183 - val_loss: 1.8729
Epoch 8/150
12500/12500 [==============================] - 24s 2ms/step - loss: 1.4342 - val_loss: 1.3649
Epoch 9/150
12500/12500 [==============================] - 23s 2ms/step - loss: 1.3109 - val_loss: 1.2083
Epoch 10/150
12500/12500 [==============================] 

KeyboardInterrupt: 